### 뉴스 content를 LLM으로 요약해서 임베딩 후 벡터 저장

In [1]:
import os
import json
import pandas as pd

news_path = "ai_news_json"
all_articles = pd.DataFrame()

# 디렉터리 내 모든 .json 파일 읽기
for filename in os.listdir(news_path):
    if filename.endswith(".json"):  # .json 파일만 처리
        file_path = os.path.join(news_path, filename)
        
        # 각 파일을 열어서 JSON 데이터 읽어오기
        with open(file_path, 'r', encoding='utf-8') as file:
            article_json = json.load(file)

            article_df = pd.DataFrame([article_json])  # JSON을 DataFrame으로 변환
            all_articles = pd.concat([all_articles, article_df], ignore_index=True)
            

# 결과 확인
print(all_articles[:3])

                                         title        created_at  \
0                        챗GPT 무단 사용 차단하는 도구 등장  2023.05.17 18:30   
1     알트먼, SF 시장 당선자 인수위원장에 임명..."머스크-트럼프와 흡사"  2024.11.19 18:00   
2  "전시 샘플이라도 내달라"...현장판매 요청에 진땀 흘린 온열 브랜드 '잉코'  2024.01.15 13:00   

                                             content  
0  '챗GPT'를 무단으로 사용하면 이를 감지해 차단해주는 보안 솔루션이 나왔다. 기업...  
1  샘 알트먼 오픈AI CEO가 대니얼 루리 차기 샌프란시스코 시장을 돕게 됐다. 이는...  
2  "전시 중인 샘플까지 떼어갈 기세라 애를 먹었습니다. 다음에는 본격 현장 판매라도 ...  


In [2]:
def save_file(txt:str, file_name:str):

    with open(file_name, 'w', encoding='utf-8') as content_file:
        content_file.write(txt)

    print(f"TEXT 파일 저장 완료: {file_name}")

In [3]:
raw_articles = all_articles.loc[:99, "content"]
raw_articles

0     '챗GPT'를 무단으로 사용하면 이를 감지해 차단해주는 보안 솔루션이 나왔다. 기업...
1     샘 알트먼 오픈AI CEO가 대니얼 루리 차기 샌프란시스코 시장을 돕게 됐다. 이는...
2     "전시 중인 샘플까지 떼어갈 기세라 애를 먹었습니다. 다음에는 본격 현장 판매라도 ...
3     LG AI 연구원이 세계 최고 수준의 '바이오 케미컬 대형언어모델(LLM)'을 만들...
4     인공지능(AI) 전문 솔트룩스(대표 이경일)는 기업 콘텐츠 관리 분야 대형언어모델(...
                            ...                        
95                AI타임스 조예주 기자 joyejuoffice@aitimes.com
96    융복합 콘텐츠 전문 상화(대표 정범준)는 서울시가 진행하는 '플레이어블 서울 프로젝...
97    1세대 의료 인공지능(AI) 전문 딥노이드(대표 최우식)는 '제50차 대한암학회 학...
98                AI타임스 조예주 기자 joyejuoffice@aitimes.com
99    이스트소프트(대표 정상원)가 올해 새롭게 론칭한 인공지능(AI) 휴먼 서비스로 성장...
Name: content, Length: 100, dtype: object

In [4]:
save_file('\n\n'.join(raw_articles), "raw_text.txt")

TEXT 파일 저장 완료: raw_text.txt


In [5]:
summerized_articles = raw_articles.copy()
summerized_articles

0     '챗GPT'를 무단으로 사용하면 이를 감지해 차단해주는 보안 솔루션이 나왔다. 기업...
1     샘 알트먼 오픈AI CEO가 대니얼 루리 차기 샌프란시스코 시장을 돕게 됐다. 이는...
2     "전시 중인 샘플까지 떼어갈 기세라 애를 먹었습니다. 다음에는 본격 현장 판매라도 ...
3     LG AI 연구원이 세계 최고 수준의 '바이오 케미컬 대형언어모델(LLM)'을 만들...
4     인공지능(AI) 전문 솔트룩스(대표 이경일)는 기업 콘텐츠 관리 분야 대형언어모델(...
                            ...                        
95                AI타임스 조예주 기자 joyejuoffice@aitimes.com
96    융복합 콘텐츠 전문 상화(대표 정범준)는 서울시가 진행하는 '플레이어블 서울 프로젝...
97    1세대 의료 인공지능(AI) 전문 딥노이드(대표 최우식)는 '제50차 대한암학회 학...
98                AI타임스 조예주 기자 joyejuoffice@aitimes.com
99    이스트소프트(대표 정상원)가 올해 새롭게 론칭한 인공지능(AI) 휴먼 서비스로 성장...
Name: content, Length: 100, dtype: object

In [6]:
import os
from dotenv import load_dotenv

load_dotenv() 
api_key = os.getenv("OPEN_API_KEY")

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(temperature=0, model="gpt-4o-mini", api_key=api_key)

In [8]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

def make_summerized_content(content:str) -> str:
    system_message = SystemMessage(content="You are a summerizing AI. Please answer in Korean. Please reply only based inputs.")
    messages = [system_message]

    messages.append(HumanMessage(content=content))
    response = model.invoke(messages)
    
    reply = response.content
    messages.append(AIMessage(content=reply))

    return reply

In [9]:
for i in range(len(summerized_articles)):
    summerized_articles[i] = make_summerized_content(summerized_articles[i])

summerized_articles

0     챗GPT의 무단 사용을 감지하고 차단하는 보안 솔루션 '트러스트 제로'가 출시됐다....
1     샘 알트먼 오픈AI CEO가 대니얼 루리 차기 샌프란시스코 시장의 인수위원회 공동 ...
2     파루 인쇄전자는 CES 2024에서 프리미엄 온열 브랜드 '잉코(INKO)' 제품을...
3     LG AI 연구원이 세계 최고 수준의 '바이오 케미컬 대형언어모델(LLM)' 개발을...
4     인공지능 전문 기업 솔트룩스가 문서중앙화 전문 기업 사이버다임과 업무협약을 체결했다...
                            ...                        
95    조예주 기자는 AI타임스에서 활동하고 있으며, 이메일 주소는 joyejuoffice...
96    융복합 콘텐츠 전문 상화(대표 정범준)는 서울시의 '플레이어블 서울 프로젝트'에서 ...
97    딥노이드가 '제50차 대한암학회 학술대회(KCA 2024)'에서 최우수 포스터상을 ...
98    조예주 기자는 AI타임스에서 활동하고 있으며, 이메일 주소는 joyejuoffice...
99    이스트소프트가 AI 휴먼 서비스로 성장세를 이어가고 있으며, 2023년 2분기 매출...
Name: content, Length: 100, dtype: object

In [10]:
save_file('\n\n'.join(summerized_articles), "summerized_text.txt")

TEXT 파일 저장 완료: summerized_text.txt


In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

def get_retriever(texts:str):

    # text_list를 Document 객체로 변환
    documents = [Document(page_content=texts)]

    recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
    )

    splits_recur = recursive_text_splitter.split_documents(documents)
    splits = splits_recur

    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

    return vectorstore.as_retriever()


In [14]:
raw_retriever = get_retriever('\n\n'.join(raw_articles))
summerized_retriever = get_retriever('\n\n'.join(summerized_articles))

In [15]:
print("================= retriever 불러오기 완료  ===============")

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(temperature=0, model="gpt-4o-mini", api_key=api_key)

contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", """
     Answer the question using only the following context.
     """),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])


class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
    
    
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
raw_rag_chain_debug = {
    "context": raw_retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model


# RAG 체인에서 각 단계마다 DebugPassThrough 추가
summerized_rag_chain_debug = {
    "context": summerized_retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model

print("================= rag_chain 불러오기 완료  ===============")

while True:
    query = input("질문을 입력하세요! 종료를 원한다면 exit을 입력하세요.")
    if query == "exit":
        break
    print("question: " + query)
    
    response = summerized_rag_chain_debug.invoke(query)
    print("RAG response : " + response.content)

================= retriever 불러오기 완료  ===============
================= rag_chain 불러오기 완료  ===============
question: 대한민국 교육박람회에 대해 알려줘
Debug Output: 대한민국 교육박람회에 대해 알려줘
Debug Output: {'context': [Document(metadata={}, page_content="제21회 대한민국 교육박람회가 17일 서울 강남구 코엑스에서 시작되었으며, 19일까지 진행된다. 이번 행사에는 420개사가 참여하고, 아시아 최대 규모로 교육 트렌드와 에듀테크를 소개한다. '교육이 미래다'라는 주제로 AI, 메타버스, XR 기술이 접목된 다양한 솔루션이 선보였다. 구글의 '클래스룸'은 AI를 활용해 교사의 수업 준비와 학생 피드백을"), Document(metadata={}, page_content='산업통상자원부는 판교 한국반도체산업협회에서 반도체아카데미 입교식을 개최했다고 밝혔다. 이번 행사에는 100여 명의 산학 관계자와 교육생이 참석했으며, 아카데미는 반도체 인력 부족 해소를 위해 실습 중심의 교육과정을 운영한다. 지난해 12월 출범한 아카데미는 현재까지 600명이 온라인 교육을 수강 중이며, 오프라인 심화 과정도 진행 중이다. 산업부 국장은'), Document(metadata={}, page_content='과정도 진행 중이다. 산업부 국장은 우수 인재 양성이 중요하다고 강조하며, 기업과 대학 간 협업의 필요성을 언급했다.'), Document(metadata={}, page_content="무대로 변신한다. 놀이존은 광화문 광장에 '빛의 놀이터'를 조성하며, 디지털 라이팅 기술을 활용한 전통 놀이 체험 콘텐츠를 제공한다. 정범준 대표는 서울의 즐거움과 매력을 알릴 기회로 삼겠다고 밝혔다.")], 'question': '대한민국 교육박람회에 대해 알려줘'}
RAG response : 제21회 대한민국 교육박람회는 17일 서울 